<a href="https://colab.research.google.com/github/shefali-netizen/Image-Description-Generation-Msc-Project/blob/main/Image_Description_Geneartion_Model_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
from google.colab import drive
from pickle import load
#from keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet101
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from os import listdir
from pickle import dump
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
import numpy as np

# extract features from each photo in the directory
def extract_features(directory):
    
	# load the model
	model = ResNet101()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# summarize
	print(model.summary())
	# extract features from each photo
	features = dict()
    
	for name in listdir(directory):
		# load an image from file
		filename = directory + '/' + name
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
        #print(features['1015118661_980735411b'])
	
	return features

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping
 
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)
 
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc
 
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

#---------------------saving image features into a pickle file------------------

drive.mount('/content/drive')

directory = 'drive/My Drive/Flicker8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('drive/My Drive/Resnet101/features.pkl', 'wb'))

#-------------------------------------------------------------------------------

#-----------------description cleaning and vocabulary creation------------------

filename = 'drive/My Drive/Flickr8k_text/Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
descriptions = load_descriptions(doc)
print('Total number of descriptions from whole dataset: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, 'descriptions.txt')

#-------------------------------------------------------------------------------

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
'''
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
  features = {k: all_features[k] for k in dataset}
	# filter features
	for k in filename:
    print(k)
    features = {k: all_features[k] }
  #features = {k: all_features[k] for k in dataset}
	return features
'''
def load_photo_features(filename, dataset):
	# load all features
  all_features = load(open(filename, 'rb'))
  #features = {k: all_features[k] for k in dataset}
	# filter features
  #print(dataset,dataset['1015118661_980735411b'])
  '''for k in dataset:
    
    #print(k)
    features = {k: all_features[k] }
    if (k=='1015118661_980735411b'):
        print("jerererre",features)'''
  features = {k: all_features[k] for k in dataset}
  #print("========",features)
  #print("here",features['1015118661_980735411b'])
  return features


#---------------------getting train images and train descriptions--------------
# load training dataset (around 6K)
filename = 'drive/My Drive/Flickr8k_text/Flickr_8k.trainImages2.txt'
train = load_set(filename)
print('Train Dataset length: %d' % len(train))
# load it's descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
# this returns s dict for example --- {image1: ['startseq girl playing','startseq child playing endseq']}
print('No. of Descriptions: train=%d' % len(train_descriptions))
# photo features
pick_read = open('drive/My Drive/Resnet101/features.pkl','rb')
data = load(pick_read)
pick_read.close()
#print("+=================",data[''])
train_features = load_photo_features('drive/My Drive/features.pkl', train)
print('Photos features length: train=%d' % len(train_features))
#print('Photos: train=%d' % train_features['1015118661_980735411b'])
#print(train_features)

#-------------------------------------------------------------------------------


def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):

 X1, X2, y = list(), list(), list()
 count=1
 # walk through each image identifier
 for key, desc_list in descriptions.items():
  # walk through each description for the image
  #print(photos.keys())
  for desc in desc_list:
   # encode the sequence
   seq = tokenizer.texts_to_sequences([desc])[0]
   # split one sequence into multiple X,y pairs
   
   for i in range(1, len(seq)):
    # split into input and output pair
    in_seq, out_seq = seq[:i], seq[i]
    # pad input sequence
    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
    # encode output sequence
    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
    # store
    
    '''if(key in photos.keys()):
        #print(key)
        count=count+1    
        #print(count)        
        X1.append(photos[key][0])
        X2.append(in_seq)
        y.append(out_seq)'''
    X1.append(photos[key][0])
    X2.append(in_seq)
    y.append(out_seq)
    '''X1 = np.array(X1.append(photos[key]))
    X2 = np.array(X2.append(in_seq))
    y = np.array(y.append(out_seq))'''
 X1 = np.array(X1)
 X2 = np.array(X2)
 y = np.array(y)
 
 
 return X1, X2, y
 
 
 
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	print(model.summary())
	plot_model(model, to_file='model.png', show_shapes=True)
	return model
 
# train dataset
 
# load training dataset (6K)
filename = 'drive/My Drive/Flickr8k_text/Flickr_8k.trainImages2.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('drive/My Drive/features.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)
 
# dev dataset
 
# load test set
filename = 'drive/My Drive/Flickr8k_text/Flickr_8k.devImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('drive/My Drive/features.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare sequences
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)
 
# fit model
 
# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = 'drive/My Drive/models/Resnet101/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
print(len(X1train))
print(len(X2train))
print(len(ytrain))

print(len(X1test))
print(len(X2test))
print(len(ytest))

model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
179658752/179648224 [==============================] - 3s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
9557/9557 - 1336s - loss: 3.8479 - val_loss: 3.9100

Epoch 00002: val_loss improved from 4.05683 to 3.91000, saving model to drive/My Drive/models/Resnet101/model-ep002-loss3.848-val_loss3.910.h5
Epoch 3/20
9557/9557 - 1320s - loss: 3.6316 - val_loss: 3.8696

Epoch 00003: val_loss improved from 3.91000 to 3.86958, saving model to drive/My Drive/models/Resnet101/model-ep003-loss3.632-val_loss3.870.h5
Epoch 4/20
9557/9557 - 1305s - loss: 3.5263 - val_loss: 3.8812

Epoch 00004: val_loss did not improve from 3.86958
Epoch 5/20
9557/9557 - 1302s - loss: 3.4640 - val_loss: 3.8850

Epoch 00005: val_loss did not improve from 3.86958
Epoch 6/20
9557/9557 - 1295s - loss: 3.4233 - val_loss: 3.9118

Epoch 00006: val_loss did not improve from 3.86958
Epoch 7/20
9557/9557 - 1294s - loss: 3.3938 - val_loss: 3.9302

Epoch 00007: val_loss did not improve from 3.86958
Epoch 8/20
9557/9557 - 1295s - loss: 3.3724 - val_loss: 3.9943

Epoch 00008: val_loss did not improve from 3.86958
Epoch 9/20
